<a href="https://colab.research.google.com/github/tlysenko/bionformatics-coursera-sandiego/blob/main/Genome%20Sequencing%20(Bioinformatics%20II)/Bioinformatics_2_week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing libraries

In [10]:
import numpy as np
import pandas as pd

from google.colab import files

import itertools
import random

from collections import defaultdict

import urllib.request

import matplotlib as plt


## 1.1.1 Cyclopeptide Scoring Problem: Compute the score of a cyclic peptide against a spectrum.

Input: An amino acid string Peptide and a collection of integers Spectrum.

Output: The score of Peptide against Spectrum, Score(Peptide, Spectrum).

In [14]:
def Cyclospectrum(peptide):
  def LinearSpectrum(peptide, alphabet, aminoAcidMass):
    
    lPeptide = len(peptide)
    prefixMass = [0] * (lPeptide) 
    for i in range(0, lPeptide):
      for symbol in alphabet:
        if symbol == peptide[i]:
          prefixMass[i] = prefixMass[i-1] + aminoAcidMass[symbol]
    prefixMass.insert(0,0)
    linearSpectrum = [0]

    c = 0
    for i in range(0, len(prefixMass)-1):
      for j in range(i+1, len(prefixMass)):
        c +=1
        linearSpectrum.append(prefixMass[j]-prefixMass[i])

    linearSpectrum.sort()
    return linearSpectrum

  # Download the data
  with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
    tb = f.readlines()
  massTable = dict()
  for line in tb:
    massTable[line.split('\n')[0].split(' ')[0]] = int(line.split('\n')[0].split(' ')[1])
    
  # *** Main ***  

  part1 = LinearSpectrum(peptide=peptide[:-1], alphabet=massTable.keys(), aminoAcidMass=massTable)
  pref1 = LinearSpectrum(peptide=peptide, alphabet=massTable.keys(), aminoAcidMass=massTable)
  tMass=pref1[-1] # mass of the full element
  part2 = [tMass - x for x in part1]

  final = part1 + part2
  final.sort()
  return final

Cyclospectrum(peptide='LEQN')

[0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]

In [41]:
def Score(peptide, spectrum):
  db = GetAminoAcidMassTable()
  CS = Cyclospectrum(peptide)
  CP = spectrum.copy()
  score = 0
  for s in CS:
    if (s in CP):
      score += 1 
      CP.remove(s)
  return score

peptide = 'NQEL'
spectrum = '0 99 113 114 128 227 257 299 355 356 370 371 484'
spectrum = [int(s) for s in spectrum.split(' ')]
Score(peptide, spectrum)

11

### Extra Dataset

In [29]:
url = 'http://bioinformaticsalgorithms.com/data/extradatasets/antibiotics/cyclopeptide_scoring.txt'
with urllib.request.urlopen(url) as f:
    data = f.read().decode('utf-8')


'Input\r\nVYYEVDWTMGRQIDPDEYPIAQCTRHRATILTLPDWQM\r\n0 71 71 87 87 97 97 99 101 101 101 101 103 113 113 113 113 113 115 115 115 115 128 128 129 129 131 131 137 147 156 156 156 163 163 163 163 172 184 186 186 199 204 210 212 212 214 214 214 216 218 226 227 228 230 231 232 243 244 244 257 259 260 260 262 273 276 278 278 281 285 287 292 293 293 301 302 303 312 314 319 325 326 327 327 327 327 328 331 332 341 358 360 364 373 374 374 375 377 389 390 391 393 394 398 402 403 407 407 409 415 416 418 425 428 429 440 440 441 441 444 445 449 456 464 465 472 475 488 490 492 494 497 499 502 503 503 504 504 505 505 512 516 517 520 521 521 531 540 541 544 550 554 556 559 560 569 570 572 573 577 578 587 592 592 601 603 605 612 613 617 618 618 619 621 621 622 625 628 634 634 653 655 657 659 661 669 671 672 675 678 680 684 688 691 696 701 704 705 707 714 715 716 716 718 721 722 723 724 732 734 735 743 748 749 749 756 768 769 775 776 781 781 785 791 792 799 800 804 806 808 809 816 819 822 822 825 829 829 8

In [35]:
peptide = data.split('\r\n')[1]
spectrum = [int(s) for s in data.split('\r\n')[2].split(' ')]
print(peptide)
print(spectrum[:10])

print('score: ', Score(peptide, spectrum))

VYYEVDWTMGRQIDPDEYPIAQCTRHRATILTLPDWQM
[0, 71, 71, 87, 87, 97, 97, 99, 101, 101]
score:  521


### Stepik submission

In [36]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_102_3-3.txt to dataset_102_3-3.txt
User uploaded file "dataset_102_3-3.txt" with length 7925 bytes


In [37]:
with open(fn) as f:
  peptide = f.readline().split('\n')[0]
  spectr = f.readline().split('\n')[0]
spectr = [int(s) for s in spectr.split(' ')]

print(spectr)
print(peptide)
print('score: ', Score(peptide, spectr))

[0, 71, 71, 87, 87, 97, 97, 99, 99, 99, 101, 101, 101, 103, 103, 113, 113, 114, 114, 115, 115, 115, 115, 128, 128, 128, 129, 131, 131, 137, 137, 137, 147, 147, 147, 147, 147, 156, 156, 163, 168, 172, 186, 186, 198, 198, 201, 202, 204, 208, 212, 213, 218, 218, 218, 224, 227, 228, 229, 229, 232, 246, 246, 248, 250, 252, 260, 260, 262, 262, 265, 268, 269, 269, 270, 271, 275, 284, 284, 291, 295, 299, 299, 300, 301, 301, 303, 316, 317, 319, 329, 332, 333, 339, 355, 355, 355, 355, 357, 358, 359, 360, 361, 363, 364, 365, 366, 370, 376, 378, 383, 385, 390, 390, 392, 396, 398, 398, 404, 412, 415, 416, 416, 418, 418, 426, 430, 431, 432, 438, 446, 447, 454, 454, 458, 460, 461, 464, 467, 470, 472, 483, 491, 492, 493, 495, 495, 497, 497, 502, 505, 511, 513, 517, 518, 519, 525, 527, 529, 532, 532, 533, 534, 545, 545, 547, 551, 553, 555, 559, 561, 561, 562, 567, 568, 571, 582, 584, 592, 594, 598, 601, 601, 607, 614, 616, 618, 619, 619, 620, 620, 624, 626, 630, 640, 647, 648, 649, 655, 655, 656, 656, 

## 1.1.2 LeaderboardCyclopeptideSequencing

Code Challenge: Implement LeaderboardCyclopeptideSequencing.

Input: An integer N and a collection of integers Spectrum.
Output: LeaderPeptide after running LeaderboardCyclopeptideSequencing(Spectrum, N).


ToDO:

- Trim function and issue with connected trims. Connected means has similar score
- Change scoring from cycle to linear

In [ ]:
# LeaderboardCyclopeptideSequencing(Spectrum, N)
#     Leaderboard ← set containing only the empty peptide
#     LeaderPeptide ← empty peptide
#     while Leaderboard is non-empty
#         Leaderboard ← Expand(Leaderboard)
#         for each Peptide in Leaderboard
#             if Mass(Peptide) = ParentMass(Spectrum)
#                 if Score(Peptide, Spectrum) > Score(LeaderPeptide, Spectrum)
#                     LeaderPeptide ← Peptide
#             else if Mass(Peptide) > ParentMass(Spectrum)
#                 remove Peptide from Leaderboard
#         Leaderboard ← Trim(Leaderboard, Spectrum, N)
#     output LeaderPeptide

In [47]:
def LeaderBoard(n, spectrum):

  def LinearSpectrum(peptide):
    #print('peptide', peptide)
    
    lPeptide = len(peptide)
    prefixMass = [0] * (lPeptide) 

    for i in range(0, lPeptide):
      for symbol in tbAAtoMass.keys():
        if symbol == peptide[i]:
          prefixMass[i] = prefixMass[i-1] + tbAAtoMass[symbol]
    prefixMass.insert(0,0)
    
    linearSpectrum = [0]
    c = 0
    for i in range(0, len(prefixMass)-1):
      for j in range(i+1, len(prefixMass)):
        c +=1
        linearSpectrum.append(prefixMass[j]-prefixMass[i])

    linearSpectrum.sort()
    #print(linearSpectrum)
    return linearSpectrum


  def GetAminoAcidMassTable():
    ''' Returns dictionary in format aminoacid: integer mass  
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[line.split('\n')[0].split(' ')[0]] = int(line.split('\n')[0].split(' ')[1])
    return massTable


  def GetMassAminoAcidTable():
    ''' Returns dictionary in format integer mass : aminoacid
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[int(line.split('\n')[0].split(' ')[1])] = line.split('\n')[0].split(' ')[0]
    return massTable


  def Expand(candidatePeptides):
      ''' Returns newCollection containing all possible extensions of peptides in 
          candidatePeptides by a single amino acid mass
      ''' 
      newCollection = set()
      if candidatePeptides != {''}:
        
        [newCollection.add(candidate + '-' + str(tbAAtoMass[peptide])) 
                          for candidate in candidatePeptides \
                            for peptide in tbAAtoMass \
                              if (tbAAtoMass[peptide] in spectrum) ]
      else:
        [newCollection.add(candidate + str(tbAAtoMass[peptide])) \
                          for candidate in candidatePeptides \
                          for peptide in tbAAtoMass \
                            if (tbAAtoMass[peptide] in spectrum)]
                        
      return newCollection


  def GetParentMass(spectrum):
    
    return spectrum[-1]
  

  def GetMass(peptide):
    ''' Input: subpeptide string
        Output: mass of subpeptidestring
    '''
    mass = 0 
    
    for element in peptide.split('-'):
      if element != '':
        mass = mass + int(element)

    return mass 


  def PeptToAA(peptide):
    ''' input: aminoacid sequence in format 137-57-128
        output: aminoacid
        TODO: combinations with similar AAs 
        113:'I/L',
        128: 'K/Q'
    '''
    aminoAcid = ''
    for aa in peptide.split('-'):
      aminoAcid += tbMasstoAA[int(aa)]
    return aminoAcid 


  def PeptideConsistentWithSpectrum(peptide):
    ''' case 1: peptide is string '128'
        case 2: peptide is string '128-113-99'
    '''
    peptideAA = PeptToAA(peptide) 
    LS = LinearSpectrum(peptide=peptideAA, alphabet=massTable.keys(), aminoAcidMass=massTable)

    spectrumCopy = spectrum.copy()
    cnt = 0

    for element in LS:
      if int(element) in spectrumCopy:
        cnt += 1 
        spectrumCopy.remove(int(element))
      else:
        return False
    if cnt == len(LS):
      return True
      

  def LinearScore(peptide, spectrum):
    LS = LinearSpectrum(peptide)
    CP = spectrum.copy()
    score = 0
    #print('LS', peptide, LS)

    if peptide != '':
      for s in LS:
        if (s in CP):
          score += 1 
          CP.remove(s)

    return score


  def Trim(leaderBoard, spectrum, n):
    ''' leaderBoard: set of peptides {'99-99-99', ''}
        spectrum: list of int [0,99,128]
        n = 10 
    '''

    if len(leaderBoard) > n:
      scores = []
      trimedLeaderBoard = set()

      [scores.append(LinearScore(PeptToAA(peptide), spectrum)) for peptide in leaderBoard]
      #print('scores', scores)
      
      sortedLB = [l for _,l in sorted(zip(scores,leaderBoard))]
      #print('sortedLB', sortedLB)
      
      scores = sorted(scores)
      #print('sorted scores', scores)

      part1 = sortedLB[-n:]
      #print('part1', part1)
      
      part2 = []
      i = 1
      while len(scores)>(n+i) and (scores[-n]==scores[-n-i]):
        part2.append(sortedLB[-n-i])
        i += 1
      trimedLeaderBoard = set(part1 + part2)
    else: 
      trimedLeaderBoard = leaderBoard
    #print('trimedLeaderBoard ', len(leaderBoard), '->',len(trimedLeaderBoard),  trimedLeaderBoard )
    return trimedLeaderBoard 


  # *** Main *** 

  tbAAtoMass = GetAminoAcidMassTable()
  tbMasstoAA = GetMassAminoAcidTable()
  parentMass = GetParentMass(spectrum)

  leaderBoard = {''} # set of strings
  leaderPeptide = ''

  while len(leaderBoard) > 0:
    leaderBoardExp = Expand(leaderBoard)
    leaderBoardCopy = leaderBoardExp.copy()

    for peptide in leaderBoardExp:  # peptide 137-57-128
      
      peptideMass = GetMass(peptide)
      if peptideMass == parentMass:
        peptideAA = PeptToAA(peptide)
        LS1 = LinearScore(peptideAA, spectrum)
        LS2 = LinearScore(leaderPeptide, spectrum)
        if  LS1 > LS2 :
          leaderPeptide = peptide 
      elif peptideMass > parentMass:
        leaderBoardCopy.remove(peptide)
    leaderBoard = Trim(leaderBoardCopy, spectrum, n)

  return leaderPeptide

n = 10 
spectrum = [int(s) for s in '0 71 113 129 147 200 218 260 313 331 347 389 460'.split(' ')]
LB = LeaderBoard(n, spectrum)
print(LB)

129-71-113-147


### Extra dataset

In [48]:
url = 'http://bioinformaticsalgorithms.com/data/extradatasets/antibiotics/leaderboard_cyclopeptide_sequencing.txt'
with urllib.request.urlopen(url) as f:
    data = f.read().decode('utf-8')

In [49]:
data.split('\r\n')[4]

'97-129-97-147-99-71-186-71-113-163-115-71-113-128-103-87-128-101-137-163-114'

In [50]:
n1 = int(data.split('\r\n')[1])
spectrum1 = [int(s) for s in data.split('\r\n')[2].split(' ')]
print(n1, spectrum1[:10])

325 [0, 71, 71, 71, 87, 97, 97, 99, 101, 103]


In [52]:
l = LeaderBoard(n1, spectrum1)
l

'99-147-97-129-97-114-163-137-101-128-87-103-128-113-71-115-163-113-71-115-71-71'

### Stepik submission

In [54]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_102_8-2.txt to dataset_102_8-2.txt
User uploaded file "dataset_102_8-2.txt" with length 2362 bytes


In [55]:
with open(fn) as f:
  n = int(f.readline().split('\n')[0])
  spectr = f.readline().split('\n')[0]
spectr = [int(s) for s in spectr.split(' ')]

print(spectr)
print(n)

[0, 71, 87, 87, 97, 99, 113, 115, 115, 128, 128, 128, 128, 128, 131, 137, 137, 147, 147, 147, 147, 163, 186, 186, 202, 202, 214, 215, 215, 218, 234, 241, 243, 246, 252, 256, 257, 260, 265, 283, 284, 289, 291, 291, 294, 294, 314, 330, 330, 333, 346, 351, 361, 369, 378, 378, 380, 380, 381, 383, 385, 388, 388, 407, 417, 419, 420, 430, 441, 442, 458, 467, 475, 479, 480, 480, 493, 495, 498, 498, 506, 506, 509, 513, 516, 516, 535, 554, 555, 567, 567, 577, 580, 582, 593, 594, 595, 595, 603, 603, 621, 621, 626, 626, 627, 635, 637, 644, 663, 666, 681, 682, 702, 708, 708, 710, 714, 714, 724, 724, 731, 731, 732, 740, 741, 749, 757, 758, 763, 766, 773, 781, 795, 809, 810, 813, 828, 836, 837, 839, 844, 859, 860, 861, 861, 868, 873, 878, 886, 894, 894, 904, 910, 918, 920, 928, 956, 960, 965, 965, 967, 972, 972, 973, 974, 975, 981, 991, 996, 996, 1001, 1008, 1022, 1022, 1038, 1046, 1051, 1062, 1065, 1067, 1075, 1088, 1093, 1093, 1096, 1100, 1102, 1104, 1107, 1109, 1119, 1119, 1121, 1135, 1138, 1150, 

In [56]:
l = LeaderBoard(n, spectr)

In [57]:
l

'97-137-147-99-115-137-128-115-87-128-163-128-87-131-71-186-128-128-113-147-147-147-99-87'

### Spectrum25 for Tyrocidine B1

In [79]:
def LeaderBoardTop(n, spectrum):

  def LinearSpectrum(peptide):
    #print('peptide', peptide)
    
    lPeptide = len(peptide)
    prefixMass = [0] * (lPeptide) 

    for i in range(0, lPeptide):
      for symbol in tbAAtoMass.keys():
        if symbol == peptide[i]:
          prefixMass[i] = prefixMass[i-1] + tbAAtoMass[symbol]
    prefixMass.insert(0,0)
    
    linearSpectrum = [0]
    c = 0
    for i in range(0, len(prefixMass)-1):
      for j in range(i+1, len(prefixMass)):
        c +=1
        linearSpectrum.append(prefixMass[j]-prefixMass[i])

    linearSpectrum.sort()
    #print(linearSpectrum)
    return linearSpectrum


  def GetAminoAcidMassTable():
    ''' Returns dictionary in format aminoacid: integer mass  
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[line.split('\n')[0].split(' ')[0]] = int(line.split('\n')[0].split(' ')[1])
    return massTable


  def GetMassAminoAcidTable():
    ''' Returns dictionary in format integer mass : aminoacid
    '''
    with open('/content/drive/MyDrive/Colab Notebooks/Bionformatics-coursera-sandiego/Course 2/integer_mass_table.txt') as f:
      tb = f.readlines()
    massTable = dict()
    for line in tb:
      massTable[int(line.split('\n')[0].split(' ')[1])] = line.split('\n')[0].split(' ')[0]
    return massTable


  def Expand(candidatePeptides):
      ''' Returns newCollection containing all possible extensions of peptides in 
          candidatePeptides by a single amino acid mass
      ''' 
      newCollection = set()
      if candidatePeptides != {''}:
        
        [newCollection.add(candidate + '-' + str(tbAAtoMass[peptide])) 
                          for candidate in candidatePeptides \
                            for peptide in tbAAtoMass \
                              if (tbAAtoMass[peptide] in spectrum) ]
      else:
        [newCollection.add(candidate + str(tbAAtoMass[peptide])) \
                          for candidate in candidatePeptides \
                          for peptide in tbAAtoMass \
                            if (tbAAtoMass[peptide] in spectrum)]
                        
      return newCollection


  def GetParentMass(spectrum):
    
    return spectrum[-1]
  

  def GetMass(peptide):
    ''' Input: subpeptide string
        Output: mass of subpeptidestring
    '''
    mass = 0 
    
    for element in peptide.split('-'):
      if element != '':
        mass = mass + int(element)

    return mass 


  def PeptToAA(peptide):
    ''' input: aminoacid sequence in format 137-57-128
        output: aminoacid
        TODO: combinations with similar AAs 
        113:'I/L',
        128: 'K/Q'
    '''
    aminoAcid = ''
    for aa in peptide.split('-'):
      aminoAcid += tbMasstoAA[int(aa)]
    return aminoAcid 


  def PeptideConsistentWithSpectrum(peptide):
    ''' case 1: peptide is string '128'
        case 2: peptide is string '128-113-99'
    '''
    peptideAA = PeptToAA(peptide) 
    LS = LinearSpectrum(peptide=peptideAA, alphabet=massTable.keys(), aminoAcidMass=massTable)

    spectrumCopy = spectrum.copy()
    cnt = 0

    for element in LS:
      if int(element) in spectrumCopy:
        cnt += 1 
        spectrumCopy.remove(int(element))
      else:
        return False
    if cnt == len(LS):
      return True
      

  def LinearScore(peptide, spectrum):
    LS = LinearSpectrum(peptide)
    CP = spectrum.copy()
    score = 0
    #print('LS', peptide, LS)

    if peptide != '':
      for s in LS:
        if (s in CP):
          score += 1 
          CP.remove(s)

    return score


  def Trim(leaderBoard, spectrum, n):
    ''' leaderBoard: set of peptides {'99-99-99', ''}
        spectrum: list of int [0,99,128]
        n = 10 
    '''

    if len(leaderBoard) > n:
      scores = []
      trimedLeaderBoard = set()

      [scores.append(LinearScore(PeptToAA(peptide), spectrum)) for peptide in leaderBoard]
      print('scores', sorted(scores,reverse=True))
      
      sortedLB = [l for _,l in sorted(zip(scores,leaderBoard))]
      #print('sortedLB', sortedLB)
      
      scores = sorted(scores)
      #print('sorted scores', scores)

      part1 = sortedLB[-n:]
      #print('part1', part1)
      
      part2 = []
      i = 1
      while len(scores)>(n+i) and (scores[-n]==scores[-n-i]):
        part2.append(sortedLB[-n-i])
        i += 1
      trimedLeaderBoard = set(part1 + part2)
    else: 
      trimedLeaderBoard = leaderBoard
    print('trimedLeaderBoard ', len(leaderBoard), '->',len(trimedLeaderBoard),  trimedLeaderBoard )
    return trimedLeaderBoard 


  # *** Main *** 

  tbAAtoMass = GetAminoAcidMassTable()
  tbMasstoAA = GetMassAminoAcidTable()
  parentMass = GetParentMass(spectrum)

  leaderBoard = {''} # set of strings
  leaderPeptide = ''

  while len(leaderBoard) > 0:
    leaderBoardExp = Expand(leaderBoard)
    leaderBoardCopy = leaderBoardExp.copy()

    for peptide in leaderBoardExp:  # peptide 137-57-128
      
      peptideMass = GetMass(peptide)
      if peptideMass == parentMass:
        peptideAA = PeptToAA(peptide)
        LS1 = LinearScore(peptideAA, spectrum)
        LS2 = LinearScore(leaderPeptide, spectrum)
        if  LS1 > LS2 :
          leaderPeptide = peptide 
      elif peptideMass > parentMass:
        leaderBoardCopy.remove(peptide)
    leaderBoard = Trim(leaderBoardCopy, spectrum, n)

  return leaderPeptide

n = 10 
spectrum = [int(s) for s in '0 71 113 129 147 200 218 260 313 331 347 389 460'.split(' ')]
LB = LeaderBoardTop(n, spectrum)
print(LB)

trimedLeaderBoard  4 -> 4 {'113', '71', '129', '147'}
scores [4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2]
trimedLeaderBoard  16 -> 12 {'147-129', '113-129', '71-113', '113-147', '147-113', '113-71', '71-129', '129-147', '147-71', '71-147', '129-113', '129-71'}
scores [7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
trimedLeaderBoard  48 -> 20 {'147-113-129', '147-113-71', '71-129-113', '113-147-71', '147-129-71', '129-71-147', '147-71-113', '113-129-71', '147-71-129', '71-129-147', '71-113-147', '129-147-71', '129-71-113', '71-147-113', '113-71-129', '129-147-113', '71-147-129', '113-71-147', '113-147-129', '129-113-147'}
scores [11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
trimedLeaderBoard  50 -> 12 {'113-147-71-129', '129-113-147-71', '147-113-129-71', '113-129-71-147',

In [77]:
url = 'http://bioinformaticsalgorithms.com/data/realdatasets/Antibiotics/Tyrocidine_B1_Spectrum_25.txt'
with urllib.request.urlopen(url) as f:
    data = f.read().decode('utf-8')

data = [int(x) for x in data.split(' ')]

data[:10]

[0, 97, 99, 113, 114, 115, 128, 128, 147, 147]

In [80]:
LBT = LeaderBoardTop(n=1000, spectrum = data)
LBT 

trimedLeaderBoard  9 -> 9 {'115', '97', '147', '99', '128', '113', '163', '186', '114'}
trimedLeaderBoard  81 -> 81 {'147-99', '97-128', '186-99', '115-114', '97-113', '99-186', '113-163', '114-99', '147-128', '113-114', '163-128', '114-97', '163-163', '99-115', '99-114', '113-128', '115-115', '113-186', '147-115', '113-147', '163-97', '113-113', '163-113', '128-113', '186-186', '163-114', '114-163', '97-147', '115-97', '128-97', '128-147', '97-186', '97-114', '115-113', '97-97', '99-147', '99-99', '128-99', '115-163', '186-113', '97-99', '99-97', '128-128', '186-128', '115-99', '115-147', '128-115', '97-163', '128-186', '114-113', '147-113', '114-128', '186-147', '163-99', '115-186', '114-115', '147-147', '147-114', '186-115', '186-163', '186-114', '186-97', '128-114', '114-114', '114-147', '114-186', '115-128', '113-97', '97-115', '163-115', '163-147', '99-163', '147-97', '113-115', '128-163', '163-186', '99-113', '147-163', '113-99', '147-186', '99-128'}
trimedLeaderBoard  729 -> 72

'186-147-114-128-99-163-128-113-97-147'

In [61]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dataset_102_10.txt to dataset_102_10.txt
User uploaded file "dataset_102_10.txt" with length 398 bytes


In [71]:
with open(fn) as f:
  data = f.readlines()

n = data[0].split('\n')[0]
spectrum = data[1].split('\n')[0]